In [1]:
import os
import cv2
import math
import numpy as np
import pandas as pd
from sklearn import svm
from scipy import signal
from matplotlib import pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score,plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedKFold

In [2]:
def cropimg(img):
    try:
        crop = cv2.cvtColor(img,cv2.COLOR_B2GRAY)
        height, width = crop.shape
        xl=0; xr=width-1; 
        for i in range(int(width/2)):
            if crop[int(height/2),i] <10:
                xl = i
            if crop[int(height/2),width-1-i] < 10:
                xr = width -1- i
        test = img[:,xl:xr]
    except:
        test = img
    return test

In [3]:
def extractionfeature(path):
    img = cv2.imread(path,0)
    img = cropimg(img)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    lab_img = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    l,a,b = cv2.split(lab_img)
    # create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cll = clahe.apply(l)
    update_img = cv2.merge((cll,a,b))
    rgb_img = cv2.cvtColor(update_img,cv2.COLOR_LAB2RGB)
    final_img = cv2.resize(rgb_img, (112,112), interpolation = cv2.INTER_AREA)
    #final_img = cv2.cvtColor(final_img,cv2.COLOR_RGB2GRAY)
    fd,hog_img = hog(final_img, orientations=8, pixels_per_cell=(8,8),cells_per_block=(4, 4),block_norm= 'L2',visualize=True,multichannel=True)
    return img ,final_img ,fd,hog_img

In [5]:
import random
path = r"C:/Users/Acer/Desktop/projectAI/Train"
os.chdir(path)
list_folder = os.listdir()
print(list_folder)
list_data = [];list_name = [];list_features = [];list_type = []
tp = 0
count = 0
for folder in list_folder:
    name_folder = path +"/" +folder
    os.chdir(name_folder)
    list_name_img = os.listdir()
    #if(len(list_name_img) > 1500):
    list_name_img = list_name_img
    for name in list_name_img:
        if (count %100 )== 0:
            print(count)
        path_img = name_folder+"/" + name
        img ,final_img ,fd,hog_img = extractionfeature(path_img)
        #list_data.append([folder+"_"+name,tp,fd,img,final_img,hog_img])
        list_name.append(folder+"_"+name)
        list_features.append(fd)
        list_type.append(tp)
        count +=1
    tp +=1
print("complete")

['Glaucoma', 'Normal', 'Other']
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
complete


In [6]:
labels = np.array(list_type).reshape(len(list_type),1)

In [7]:
features = np.array(list_features)
list_name = [];list_features = [];list_type = []
data = np.hstack((labels,features))

In [8]:
np.random.shuffle(data)

In [65]:
Y = data[:,0]
X = data[:,1:]
Y = np.array([np.int32(x) for x in Y])
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

In [ ]:
clfknn = KNeighborsClassifier(n_neighbors = 20)
clfknn.fit(x_train, y_train)
y_pred = clfknn.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))
plot_confusion_matrix(clfknn, x_test, y_test)  

In [113]:
import pickle
Pkl_Filename = "modelknn_superwow.pkl"
path = r"C:/Users/Acer/Desktop/projectAI/muteluh-fundus"
os.chdir(path)
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clfknn, file)
path = r"C:/Users/Acer/Desktop/projectAI"
os.chdir(path)
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clfknn, file)

In [ ]:
clfsvm = svm.SVC(probability=True)
clfsvm.fit(x_train,y_train)
y_pred = clfsvm.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))
plot_confusion_matrix(clfsvm, x_test, y_test)

In [ ]:
import pickle
Pkl_Filename = "modelsvm_superwow5.pkl"
path = r"C:/Users/Acer/Desktop/projectAI/muteluh-fundus"
os.chdir(path)
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clfsvm, file)
path = r"C:/Users/Acer/Desktop/projectAI"
os.chdir(path)
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clfsvm, file)

In [ ]:
def zoom(image):
    b,g,r = cv2.split(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    b = b- b.mean() -b.std()
    g = g- g.mean() -g.std()
    r = r- r.mean() -r.std()
    total = (r+g+b)/3
    
    #total[total > 2*total.std() ] = 255
    b[b < 2.5*b.std()] = 0
    b[b > 2.5*b.std()] = 255

    gray = gray - gray.mean() -gray.std()
    
    test = 1.5*total - b - gray
    test[test<0] = 0

    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(test)
    height, width, channels = image.shape
    rate = height // 5
    upper,bottom,left,right = maxLoc[1]-rate,maxLoc[1]+rate,maxLoc[0]-rate,maxLoc[0]+rate
    upper = 0 if upper<0 else upper
    bottom = 0 if bottom<0 else bottom
    left = 0 if left<0 else left
    right = 0 if right<0 else right
    crop = image[upper:bottom,left:right]
    return  cv2.resize(crop, dim, interpolation = cv2.INTER_AREA)